## Yolo11 tests - visualizations of results

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os , sys
from pathlib import Path
cur_path = Path(os.getcwd()).parent.parent
sys.path.append(cur_path.as_posix())
# Enable plotting and make plots pretty (seaborn)
from matplotlib import pyplot as plt
import matplotlib as mpl
plt.style.use('dark_background')
import seaborn as sns
sns.set_theme(style='white')
%config InlineBackend.figure_format = 'retina'
plt.style.use('dark_background')
mpl.rcParams.update({"axes.grid" : True, "grid.color": "grey", "grid.alpha": .3})
plt.rcParams['xtick.major.size'] = 10
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['ytick.major.size'] = 10
plt.rcParams['ytick.major.width'] = 1
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True

In [ ]:
from octron.yolo_octron.yolo_octron import YOLO_octron
import pandas as pd
import numpy as np
import cv2
from scipy.ndimage import distance_transform_edt

In [ ]:
yolo_octron = YOLO_octron()

In [ ]:
output_dir = Path('/Users/horst/Downloads/predictions/20221130_Behaviour_Single_3_clip_bytetrack')

In [ ]:
results_dict = {}
for label, track_id, track_df_napari, features_df_napari, mask_zarr in yolo_octron.show_predictions(save_dir = output_dir):
    print(label, track_id)
    results_dict[label] = {
        'track_id': track_id,
        'track_df_napari': track_df_napari,
        'features_df_napari': features_df_napari    
        
        
    }
    if label == 'head':
        break
    # for i, row in track_df_napari.iterrows():
    #     print(row['frame'], row['x'], row['y'])
    #     plt.imshow(row['image'])
    #     plt.scatter(row['x'], row['y'], color = 'red')
    #     plt.show()

## [WIP] Mask Interpolation

In [ ]:
from octron.yolo_octron.helpers.mask_interpolation import interpolate_masks

In [ ]:
mask = np.zeros((500,500))
mask1 = mask.copy()
mask2 = mask.copy()
mask1[30:80, 30:80] = 1 
mask2[100:160, 260:280] = 1

In [ ]:
mask_A = mask1
mask_B = mask2

interpolated_masks = interpolate_masks(mask_A, mask_B, num_frames=15, control_points=150)

In [ ]:
for i, mask in enumerate(interpolated_masks):
    plt.imshow(mask)
    plt.title(f'Interpolated Mask {i+1}')
    plt.axis('off')
    plt.show()